In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define paths to your training, validation, and testing data
train_dir = '/kaggle/input/pnevmoniya/train'
test_dir = '/kaggle/input/pnevmoniya/test'

In [ ]:
img_size = 224
num_classes = 2
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model with a loss function, optimizer, and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model on the training data and validate it on the validation data
model.fit(train_generator, epochs=10)

In [ ]:
df = pd.read_csv('../input/pnevmoniya/sample_solution.csv')
df.head()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

j = 0
for i in df['id'].values:
    img_path = f"{test_dir}/{i}"
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 225.
    prob = model.predict(x)
    if prob[0][0] < prob[0][1]:
        df.loc[df['id'] == i, 'labels'] = 1
        j += 1
        

In [ ]:
df['labels'].value_counts()

In [ ]:
df.to_csv("/kaggle/working/solution1.csv", index=False)